In [1]:
import pandas as pd
import numpy as np
import scanpy as sc
from matplotlib.pyplot import rc_context
sc.settings.verbosity = 3



In [2]:
mergeddata = sc.read("/Users/athib/Desktop/CovidRSV/scRNA2/Merged_QC.h5ad")


In [3]:
sc.pp.filter_genes(mergeddata, min_cells=5)


filtered out 8158 genes that are detected in less than 5 cells


In [4]:
mergeddata.var['mt'] = mergeddata.var_names.str.startswith('MT-')
sc.pp.calculate_qc_metrics(mergeddata, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)


In [5]:
sc.pp.highly_variable_genes(mergeddata, batch_key='batch')


extracting highly variable genes
    finished (0:00:40)
--> added
    'highly_variable', boolean vector (adata.var)
    'means', float vector (adata.var)
    'dispersions', float vector (adata.var)
    'dispersions_norm', float vector (adata.var)


In [8]:
mergeddata.obs.rename(columns={"batch": "sample"}, inplace=True)

In [9]:
metadata =pd.read_csv("/Users/athib/Desktop/CovidRSV/scRNA2/BatchCorrection_MetaData/CovidRSVScanpyMetadata.csv")

In [10]:
for curcol in metadata.columns:
    mergeddata.obs[curcol] = None
    
    for cursample in metadata['Sample ID']:
        curvalue = metadata[metadata['Sample ID'] == cursample][curcol].values[0]
        mergeddata.obs[curcol][mergeddata.obs['sample'] == cursample] = curvalue


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [11]:
mergeddata.obs['Age'] = pd.to_numeric(mergeddata.obs['Age'])

In [12]:
mergeddata.raw = mergeddata
mergeddata = mergeddata[:, mergeddata.var.highly_variable]

In [13]:
sc.pp.scale(mergeddata, max_value=10)
sc.tl.pca(mergeddata, svd_solver='arpack',n_comps=50)

/Users/athib/Library/Python/3.7/lib/python/site-packages/scanpy/preprocessing/_simple.py:843: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)


... as `zero_center=True`, sparse input is densified and may lead to large memory consumption
computing PCA
    on highly variable genes
    with n_comps=50
    finished (0:00:55)


In [16]:
import scanpy.external as sce

In [17]:
sce.pp.harmony_integrate(mergeddata, 'Batch')


2022-12-12 11:42:15,486 - harmonypy - INFO - Iteration 1 of 10
2022-12-12 11:45:14,735 - harmonypy - INFO - Iteration 2 of 10
2022-12-12 11:48:13,864 - harmonypy - INFO - Iteration 3 of 10
2022-12-12 11:50:15,740 - harmonypy - INFO - Converged after 3 iterations


In [21]:
mergeddata.obs['Batch'] = pd.to_numeric(mergeddata.obs['Batch'])

In [23]:
mergeddata.obs['Steroid'] = pd.to_numeric(mergeddata.obs['Steroid'])

In [24]:
mergeddata.write("/Users/athib/Desktop/CovidRSV/scRNA2/HarmonyCorrected.h5ad")
